## 3. 튜토리얼/금융 분석 프로그래밍 응용

<p algin="right">
파이썬 주식 시장 동향 분석 - 자연어처리 감성분석
swsong
2023. 1. 12. 07:04
</p>

<p>origin : https://songseungwon.tistory.com/125</p>

- Step 1. 블로그 정보 수집Step
- Step 2. 블로그 내용 수집Step
- Step 3. 감성 분류

<p>주식 시장에는 기본적으로 주식을 팔고자 하는 사람과 사고자 하는 사람, 이렇게 크게 두 유형이 있다. 팽팽한 줄다리기 끝에 팔고자 하는 사람이 더 많으면 공급 초과로 가격은 하락하게 된다.
​주가에 영향을 주는 요소는 금융 애널리스트의 예측, 기관 예측, 경제 위기, 그리고 이를 전달하는 뉴스 매체 등으로 셀 수 없이 많지만 결국 그 종착지에는 대중의 움직임이 있다. 그러한 대중의 움직임이 파는 쪽에 더 치우치면 가격은 떨어지고, 사는 쪽에 치우치면 가격은 상승한다.
그렇다면 우리는 지금 이 시간, 대중의 움직임이 어느 방향으로 기울고 있는가를 분석해 볼 수 있다. 그 대상은 특정 종목이나 특정 국가가 될 수도 있고 주식 시장 자체가 될 수 있다. 본 분석은 후자, 주식 시장 자체에 대해 사람들이 어떻게 평가하고 있는가를 분석해 볼 것이며 분석 대상이 되는 기간은 1주일이다. 즉, '이번 주 주식 시장에 대한 사람들의 평가는 어떠한가?'에 대한 분석이 되겠다.(2022-10-12 기준)</p>

### Step 1. 블로그 정보 수집


#### 1-1. 웹사이트 구조 및 데이터 호출 정보 확인

<p>파이썬으로 포스팅을 긁어올 것이므로 타깃을 명확히 확인하자. '네이버 블로그'(https://section.blog.naver.com/) 사이트에 진입해서 '주가 전망'이라는 키워드를 검색 후 개발자 모드를 열어 네트워크 탭을 살펴본다.
미리 보기를 통해 반환된 문서들을 살펴보면 그중 SearchList.naver에 우리가 찾는 데이터가 있는 것을 확인할 수 있다. 이제 파이썬으로 해당 자료를 가져올 수 있으면 된다.
먼저 해당 데이터가 어떤 방식(GET, POST)으로, 어떤 url 값으로 반환받을 수 있는지 살펴본 다음, 하단 쿼리 문자열 매개변수를 확인함으로써 넘겨줄 요청 값들을 지정해 줄 수 있다.</p>

<img src="./img/finance_analyze_01.png"/>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import json

#### 1-2. 단일 페이지 데이터 호출

<p>currentPage는 현재 페이지 번호, countPerPage는 페이지당 포함하는 포스팅 수, endDate와 startDate는 지난 7일간을 지정해 주고 keyword는 검색한 값을 보여준다. 이렇게 파라미터 파악이 끝났으면 코드 작성을 위해 필요한 라이브러리를 호출하겠다.</p>

<p>endDate, startDate는 직접 문자열로 입력해 줘도 되지만 스크래퍼를 개발할 때에는 항상 자동화를 염두에 두어야 한다. 따라서 기준 날짜로부터 지난 7일을 계산할 수 있도록 다음과 같이 함수를 사용하겠다.</p>

In [2]:
datetime.today() - timedelta(30)

datetime.datetime(2025, 5, 1, 23, 16, 3, 16906)

In [3]:
datetime.today().strftime('%Y-%m-%d')

'2025-05-31'

<p>이제, url, header, date, params를 정의하고 파라미터가 잘 세팅되는지 확인한다.</p>

In [4]:
url = 'https://section.blog.naver.com/ajax/SearchList.naver'
header = {
    "Referer": "https://section.blog.naver.com/Search/Post.naver",
}
end_date = datetime.today()
start_date = end_date - timedelta(days=7)
params = {
    'countPerPage': 7,
    'currentPage': 1,
    'endDate': end_date.strftime('%Y-%m-%d'),
    'keyword': '주가전망',
    'orderby': 'sim',
    'startDate': start_date.strftime('%Y-%m-%d'),
    'type' : 'post'
}
params

{'countPerPage': 7,
 'currentPage': 1,
 'endDate': '2025-05-31',
 'keyword': '주가전망',
 'orderby': 'sim',
 'startDate': '2025-05-24',
 'type': 'post'}

<p>requests 모듈을 통해 데이터를 불러온다. 결괏값을 확인했을 때 우리가 앞서 봤던 SearchList.naver의 미리 보기 값과 형태가 동일해야 한다. 텍스트 앞부분만 짧게 출력하자.</p>

In [5]:
res = requests.get(url, params=params, headers=header)
res
res.text[:1000]

')]}\',\n{"result":{"searchDisplayInfo":{"authUrlType":"LOGIN","authUrl":"https://nid.naver.com/nidlogin.login?svctype=128&a_version=2&viewtype=1&&url=https://section.blog.naver.com/Search/Post.naver?keyword=%25EC%25A3%25BC%25EA%25B0%2580%25EC%25A0%2584%25EB%25A7%259D&surl=https://section.blog.naver.com","existSuicideWord":false,"keyword":"주가전망","eucKrEncodedKeyword":"%C1%D6%B0%A1%C0%FC%B8%C1","displayType":"NORMAL","blockedByBifrostShield":false},"searchList":[{"domainIdOrBlogId":"press02","logNo":223881896022,"gdid":"90000003_00000000000000342066A056","postUrl":"https://blog.naver.com/press02/223881896022","title":"원전관련주 두산에너빌리티 목표<strong class=\\"search_keyword\\">주가 전망</strong>: 해외수주 양호","noTagTitle":"원전관련주 두산에너빌리티 목표주가 전망: 해외수주 양호","contents":"실제로 증권사들의 목표<strong class=\\"search_keyword\\">주가</strong> 평균인 38,286원은 현재가보다 낮습니다. 가장 높은 <strong class=\\"search_keyword\\">전망</strong>은 대신증권의 46,000원, 가장 낮은 곳은 유안타증권의 31,000원이었습니다. 이 말은 ‘이미 오를 만큼 오른 것 아니냐’는 우려도 있다는 뜻입니다. 게다가 현재 PER(<strong

<p>데이터를 json 형태로 변환해 주면 좋겠다. 그전에 앞부분의 불필요한 문자열을 잘라줘야 정상적으로 변환이 될 것이다.</p>

In [6]:
import json
json.loads(res.text.split('\n')[1])['result']['searchList']

[{'domainIdOrBlogId': 'press02',
  'logNo': 223881896022,
  'gdid': '90000003_00000000000000342066A056',
  'postUrl': 'https://blog.naver.com/press02/223881896022',
  'title': '원전관련주 두산에너빌리티 목표<strong class="search_keyword">주가 전망</strong>: 해외수주 양호',
  'noTagTitle': '원전관련주 두산에너빌리티 목표주가 전망: 해외수주 양호',
  'contents': '실제로 증권사들의 목표<strong class="search_keyword">주가</strong> 평균인 38,286원은 현재가보다 낮습니다. 가장 높은 <strong class="search_keyword">전망</strong>은 대신증권의 46,000원, 가장 낮은 곳은 유안타증권의 31,000원이었습니다. 이 말은 ‘이미 오를 만큼 오른 것 아니냐’는 우려도 있다는 뜻입니다. 게다가 현재 PER(<strong class="search_keyword">주가</strong>수익비율)은 64.1배, PBR(<strong class="search_keyword">주가</strong>순자산비율)은... ',
  'nickName': '재미진 저널리스트',
  'blogName': '재미진 저널리스트의 금융탐구생활',
  'profileImgUrl': 'https://blogpfthumb-phinf.pstatic.net/MjAyNDA3MjNfMjM2/MDAxNzIxNzA0NjQyMTgz.or0KzbaFI9cpwv4aAeT86_hscrGZFPCbyPT5e2asceUg.wga-Vaoidl-BtmYVO0mldLwk1NpD2qX6xqF8lDtawIsg.JPEG/1.jpeg?type=s1',
  'addDate': 1748496600000,
  'thumbnails': [{'url': 'https://blogthumb.p

#### 1-3. 다중 페이지 데이터 호출(함수)
이제, 앞서 구현한 코드들을 종합하여 키워드와 페이지 번호만 넣어주면 지난 7일간의 포스팅 정보를 긁어올 수 있는 함수를 정의하겠다.

In [7]:
def get_posts_info(keyword, start_page=1, end_page=10):
    url = 'https://section.blog.naver.com/ajax/SearchList.naver'
    headers = {
        'Referer': 'https://section.blog.naver.com/search/Post.naver',
    }
    end_date = datetime.today()
    start_date = end_date - timedelta(days=7)

    all_posts_info = []
    print(f'[INFP] keyword : {keyword}, start_page : {start_page}, end_page : {end_page}')
    for i in range(start_page, end_page+1):
        params = {
            'countPerPage': 7,
            'currentPage': 1,
            'endDate': end_date.strftime('%Y-%m-%d'),
            'keyword': keyword,
            'orderBy': 'sim',
            'startDate': start_date.strftime('%Y-%m-%d'),
            'type': 'post',
        }
        res = requests.get(url, params=params, headers=headers)
        current_posts_info = json.loads(res.text.split('\n')[1])['result']['searchList']
        all_posts_info += current_posts_info
        print(f'[INFO] 포스트 정보 수집 중 .. (page : {i}/{end_page} current posts : {len(current_posts_info)} all posts : {len(all_posts_info)}')
    return all_posts_info

In [8]:
posts_info = get_posts_info('주가전망', 1, 10)

[INFP] keyword : 주가전망, start_page : 1, end_page : 10
[INFO] 포스트 정보 수집 중 .. (page : 1/10 current posts : 7 all posts : 7
[INFO] 포스트 정보 수집 중 .. (page : 2/10 current posts : 7 all posts : 14
[INFO] 포스트 정보 수집 중 .. (page : 3/10 current posts : 7 all posts : 21
[INFO] 포스트 정보 수집 중 .. (page : 4/10 current posts : 7 all posts : 28
[INFO] 포스트 정보 수집 중 .. (page : 5/10 current posts : 7 all posts : 35
[INFO] 포스트 정보 수집 중 .. (page : 6/10 current posts : 7 all posts : 42
[INFO] 포스트 정보 수집 중 .. (page : 7/10 current posts : 7 all posts : 49
[INFO] 포스트 정보 수집 중 .. (page : 8/10 current posts : 7 all posts : 56
[INFO] 포스트 정보 수집 중 .. (page : 9/10 current posts : 7 all posts : 63
[INFO] 포스트 정보 수집 중 .. (page : 10/10 current posts : 7 all posts : 70


<p>총 10개 페이지, 70개 포스팅 정보가 수집되었다.</p>

### Step 2. 블로그 내용 수집

#### 2-1. HTML 태그 정보 확인

<p>이렇게 수집한 포스팅 정보로 각 url을 순회하며 텍스트만 뽑아올 것다. 수집한 url 중 하나에 진입해서 html 태그를 살펴보자.</p>

<img src="img/finance_analyze_02.png"/>

<p>본문 텍스트는 se-main-continer라는 div class가 지정되어 있다.</p>

#### 2-2. 데이터 호출 정보 확인

<p>데이터 호출을 위해 이번에는 네트워크 탭의 '문서'를 살펴보면 PostView.naver에 우리가 찾는 본문 텍스트가 있다. 해당 데이터를 어떻게 호출할 수 있을지 url과 매개변수를 통해 살펴본다.</p>

<img src="img/finance_analyze_03.png"/>

#### 2-3. 단일 페이지 데이터 호출(함수)

<p>이번에는 함수로 바로 만들어주겠다. select_one 함수로 본문을 가져오고 select('p')를 해줌으로써 본문 내 모든 문장들을 리스트로 추출한다. 또한, list comprehension 문법을 통해 각 리스트 내 요소에서 텍스트만 추출해 다시 리스트로 저장하고, 제어문자(u200b는 폭 없는 공백이다. 네이버 블로그 포스팅 시 자동으로 문자 사이에 포함됩니다.)는 없애준다.</p>

<p>계속해서 list comprehension 문법을 사용하며 줄바꿈은 분리해 주고 공백 요소는 제거해 깔끔한 형태의 문자열만 담은 리스트로 만든다.</p>

In [9]:
def get_posts(x):
    url = 'https://blog.naver.com/PostView.naver'
    params = {
        'blogId': x['domainIdOrBlogId'],
        'logNo': x['logNo'],
        'redirect': 'Dlog',
        'widgetTypeCall': 'true',
        'directAccess': 'false'
    }

    res = requests.get(url, params=params)
    soup = BeautifulSoup(res.text, 'lxml')
    posts = soup.select_one('.se-main-container').select('p')
    posts = [x.get_text().replace('\u200b', '') for x in posts]

    filtered_posts = [x.replace('다. ', '다. \n') for x in posts]
    filtered_posts = sum([x.split('\n') for x in posts], [])
    filtered_posts = [x.strip() for x in filtered_posts if not x in ['', ' ']]
    return filtered_posts

In [10]:
get_posts(posts_info[0])

['요즘 주식시장 보면 마치 롤러코스터를 탄 듯한 종목들이 많습니다.',
 '그중에서도 유난히 고공비행 중인 종목이 하나 있는데요, 바로 두산에너빌리티입니다.',
 '이름부터 뭔가 거창한 이 기업, 최근 주가가 하늘 높은 줄 모르고 치솟고 있습니다.',
 '도대체 무슨 일이 벌어지고 있는 걸까요?',
 '주가는 거짓말을 안 한다?',
 '한때 15,150원이었던 이 종목이 불과 9개월 만에 43,450원까지 올라갔습니다.',
 '그냥 ‘오른 게 아니라’ 거의 세 배 가까이 치솟은 셈입니다.',
 '지금은 41,650원 선에서 거래 중인데, 하루 전보다 1,850원이나 오른 상태죠.',
 '이쯤 되면 그냥 강세장이 아니라 거의 ‘폭등장’입니다.',
 '주식 좀 해보신 분들은 아시겠지만, 이런 흐름은 아무 이유 없이 나오지 않습니다.',
 '이번엔 사우디다, 그것도 두 번',
 '최근 두산에너빌리티가 사우디에서만 두 건의 수주를 따냈다는 소식이 들려왔습니다.',
 '1300억 원 규모의 연료전환 사업에 이어, 3400억 원 규모의 가스복합발전소 설비 계약까지.',
 '연이은 수주 소식은 마치 크리스마스 선물처럼 투자자들을 들뜨게 했습니다.',
 '계약 대상은 리야드 북동쪽에 위치한 발전소 두 곳인데, 증기터빈과 발전기를 각각 두 기씩 납품할 예정이라고 하니, 그 규모만 봐도 상당하죠.',
 '게다가 이 회사, 최근 5년간 전 세계에서 발주된 초대형 증기터빈 물량의 33%를 수주했다는 자랑거리도 있습니다.',
 '이쯤 되면 단순히 ‘운 좋게 딴 계약’이 아니라, 기술력으로 증명된 실력이라 봐야겠죠.',
 '투자? 그냥 하는 게 아닙니다',
 '수주만 많다고 회사가 무조건 좋아지진 않습니다.',
 '문제는 그걸 잘 만들어내느냐인데요, 두산에너빌리티는 이 부분에서도 승부수를 띄웠습니다.',
 '2027년까지 무려 1조 3,232억 원 규모의 설비 투자를 예고했습니다.',
 '올해만 4,796억 원을 쓰겠다고 했는데, 작년 2,140억 원과 비교하면 두 배가 넘는 규모입

In [11]:
get_posts(posts_info[1])

['안녕하세요 경제 인플루언서 디노!입니다.',
 '오늘 알아볼 국내주식은 에코프로 입니다.',
 '현재 가격 분석과 주가전망 그리고 하락하는 이유까지 살펴보겠습니다.',
 '에코프로 주가 현황, 가격분석',
 '에코프로 주가현황입니다.',
 '23년 전성기를 누리던 주식이 딱 2년이 지난 지금은 하염없이 하락만 이어지고 있습니다.',
 '고점에 비해서는 -85% 수준의 가격이고, 최근 2년간 제대로 된 반등 한번 없이 하락하는걸 보면 정말 테마주였나..? 하는 생각마저 드는데요.',
 '(에코프로는 24년 액면분할이 있었습니다)',
 '저도 한때 2차전지주인 엘앤에프를 보유했던 기억이 있어서 남의 일 같지가 않은 차트입니다.',
 '현재 가격이 어떤지 분석해보려고 일봉으로도 보고, 주봉으로도 봤지만 이평선 밑에서 계속 놀고있고, 한번을 이평위로 지지받지 못하는 모습을 보이고 있습니다. 이런 부분만 봐도 여전히 대중은 비싸다고 생각하는것 같습니다.',
 '에코프로의 현 시총은 5.2조로 과거 30조를 찍엇던 시기를 생각하면 싸보이지만.. 현재 상황은 이것도 비싸보입니다.',
 '과연, 현재 에코프로는 현재 어떤 상태를 보이고 있는 기업일지.. 살펴보도록 하겠습니다.',
 '에코프로 현재는 어떤가?',
 '그럼, 에코프로의 매출과 영업이익 상태부터 진단해보겠습니다.',
 '22년 매출 5.6조, 23년 7.2조를 기록하던 에코프로',
 '24년에는 50% 이상 하락한 3.1조를 기록했고 영업이익은 -2930억으로 적자전환이 이뤄졌습니다.',
 '분기 매출을 살펴봐도 이렇다할 반등도 없고, 이익 역시 한분기만의 miss가 아닌 지속적인 적자를 기록하고 있습니다.',
 '에코프로 주가 하락하는 이유, 전망은?',
 '에코프로가 이 같이 하락하는데에는 중국의 영향이 상당히 큽니다.',
 '중국은 상대적으로 가격이 낮고 효율이 좋은 LFP배터리로 점유율을 키워오고 있습니다.',
 '중국이 선택한 LFP가 배터리 시장의 표준으로 인식되고있고, 여기에 그동안 단점으로 꼽히

### Step 3. 감성 분류

<p>이제, 이렇게 얻은 문자열에 대해 감성 분류를 실시할 수 있다. 텍스트 클렌징 작업은 최대한 정교하게 할수록 좋다. 텍스트 수집에 정도가 없듯 텍스트 전처리 방법에도 역시 정도가 없다.</p>
<p>이번 튜토리얼에서는 간단하게 전처리한 문장을 활용해 감성 분류를 해보자.</p>

#### 3-1. 허깅페이스 모델 검색

<p>우선, 임의의 텍스트 한 줄을 가져다 잘 분류해 줄 수 있는 모델을 찾아보자.</p>
<p>언어 모델의 경우 대규모로 사전학습된 모델을 먼저 리서치해 보는 것이 좋다. 간단한 분석을 위해 몇 주간의 학습 시간을 소진하기에는 현실적으로 어렵기 때문에, 언어 모델 사용 시 다운스트림 테스크로 우리의 데이터 셋에 조금 더 확률 값을 높여주고 우리가 목표로 하는 결괏값을 출력해 내도록 구조를 맞춰주는 정도의 파인튜닝이 일반적이다.</p>
<p>감성 분류, 그중에서도 금융 텍스트에 특화된 한국어 모델이 이미 허깅페이스에 올라와 있다. 해당 모델을 테스트해 보고 사용 여부를 판단한다.</p>
<img src="img/finance_analyze_04.png"/>
<p>부정적 어감의 문장을 가져다 넣으니 negative로 잘 분류해 준다. 중립은 어떨까?</p>
<img src="img/finance_analyze_05.png"/>
<p>'엔비디아 전망'이라는 텍스트는 97.6%의 확률로 neutral으로 분류를 해준다. 만약 우리가 사용하는 모델이 긍/부정으로만 분류할 수 있다면 중립에 해당하는 문장은 최대한 걸러내야 할 것이다.</p>
<p>여기서는 우리가 수집한 전체 문장에 대해 중립, 부정, 긍정 각각의 비중을 살펴보고 주식시장의 동향을 파악해 보고자 한다.</p>
<p>우리는 특정 블로그 포스팅 자체에 대해 '긍정 포스팅이다.', '부정 포스팅이다.' 하고 분류하기보다 위 방식처럼 하나의 글 내에서도 여러 문장으로 분리하고 각각을 분류 대상으로 삼을 것이다. 한 명이 작성한 하나의 블로그 포스팅 안에서도 긍정적인 문장과 부정적인 문장이 혼재해있다. 해당 블로그 포스팅으로 누군가는 부정적인 정보를 획득해갈 것이고 누군가는 반대로 긍정적인 정보를 획득해간다. 사람마다 받아들이는 정보는 매우 주관적이므로 블로그 포스팅 내 모든 문장을 정보 혹은 대중의 판단 근거라 가정하는 것이다.</p>
<p>물론 이러한 분석 방식 역시 주관과 의도가 강하게 개입되어 있으며, 유일한 정답일 수 없다.</p>

#### 3-2. 추론 모델 인스턴스 생성 및 테스트

In [33]:
!pip freeze > req.txt
!pwd

/home/swhors/jupyter-workspace/LSTM/study


In [17]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained('snunlp/KR-FinBert-SC')
model = AutoModelForSequenceClassification.from_pretrained('snunlp/KR-FinBert-SC')
senti_classifier = pipeline(task='text-classification', model=model, tokenizer=tokenizer)

Device set to use cpu


<p>위 코드는 허깅페이스를 통해 우리가 사용하려는 모델과 토크나이저를 불러와 빠르게 추론 모델을 만든다.</p>
<p>해당 모델은 뉴스 기사 텍스트로 1차 파인튜닝된 모델이다. 그렇기 때문에 추가 학습 없이 우리 테스크에 바로 적용할 수 있다.</p>
<p>다만, 경우에 따라 같은 유형의 테스크라 하더라도 모델이 우리 데이터에 완전히 맞지는 않을 수 있다. 따라서 몇 가지 단어를 넣어 적절한 출력이 나오는지 한 번 더 확인하겠다.</p>

In [19]:
senti_classifier(['상승', '하락', '애플'])

[{'label': 'positive', 'score': 0.8705510497093201},
 {'label': 'negative', 'score': 0.9346328973770142},
 {'label': 'neutral', 'score': 0.5876973867416382}]

<p>주가 관련 텍스트를 학습한 탓에 상승이라는 키워드에는 positive를, 하락이라는 키워드에는 negative를, 애플과 같은 고유명사에는 neutral을 출력해 준다. 우리가 기대했던 결과값이다.</p>
<p>그럼, 이제 지난 7일간의 주식 시장에 대한 평가를 네이버 블로그에 한정해서 살펴보도록 하겠다.</p>

#### 3-3. 단일 포스팅 내 텍스트 감성 분류

In [21]:
len(posts_info)

70

<p>우리가 분석하려는 총 포스트 수는 총 70개다. 첫 번째 포스팅을 먼저 분석해 보겠다.</p>

In [22]:
get_posts(posts_info[0])

['요즘 주식시장 보면 마치 롤러코스터를 탄 듯한 종목들이 많습니다.',
 '그중에서도 유난히 고공비행 중인 종목이 하나 있는데요, 바로 두산에너빌리티입니다.',
 '이름부터 뭔가 거창한 이 기업, 최근 주가가 하늘 높은 줄 모르고 치솟고 있습니다.',
 '도대체 무슨 일이 벌어지고 있는 걸까요?',
 '주가는 거짓말을 안 한다?',
 '한때 15,150원이었던 이 종목이 불과 9개월 만에 43,450원까지 올라갔습니다.',
 '그냥 ‘오른 게 아니라’ 거의 세 배 가까이 치솟은 셈입니다.',
 '지금은 41,650원 선에서 거래 중인데, 하루 전보다 1,850원이나 오른 상태죠.',
 '이쯤 되면 그냥 강세장이 아니라 거의 ‘폭등장’입니다.',
 '주식 좀 해보신 분들은 아시겠지만, 이런 흐름은 아무 이유 없이 나오지 않습니다.',
 '이번엔 사우디다, 그것도 두 번',
 '최근 두산에너빌리티가 사우디에서만 두 건의 수주를 따냈다는 소식이 들려왔습니다.',
 '1300억 원 규모의 연료전환 사업에 이어, 3400억 원 규모의 가스복합발전소 설비 계약까지.',
 '연이은 수주 소식은 마치 크리스마스 선물처럼 투자자들을 들뜨게 했습니다.',
 '계약 대상은 리야드 북동쪽에 위치한 발전소 두 곳인데, 증기터빈과 발전기를 각각 두 기씩 납품할 예정이라고 하니, 그 규모만 봐도 상당하죠.',
 '게다가 이 회사, 최근 5년간 전 세계에서 발주된 초대형 증기터빈 물량의 33%를 수주했다는 자랑거리도 있습니다.',
 '이쯤 되면 단순히 ‘운 좋게 딴 계약’이 아니라, 기술력으로 증명된 실력이라 봐야겠죠.',
 '투자? 그냥 하는 게 아닙니다',
 '수주만 많다고 회사가 무조건 좋아지진 않습니다.',
 '문제는 그걸 잘 만들어내느냐인데요, 두산에너빌리티는 이 부분에서도 승부수를 띄웠습니다.',
 '2027년까지 무려 1조 3,232억 원 규모의 설비 투자를 예고했습니다.',
 '올해만 4,796억 원을 쓰겠다고 했는데, 작년 2,140억 원과 비교하면 두 배가 넘는 규모입

In [27]:
x = get_posts(posts_info[0])
y = [x['label'] for x in senti_classifier(x)]

df = pd.DataFrame(data={'text': x, 'senti': y})
df

,text,senti
0,요즘 주식시장 보면 마치 롤러코스터를 탄 듯한 종목들이 많습니다.,neutral
1,"그중에서도 유난히 고공비행 중인 종목이 하나 있는데요, 바로 두산에너빌리티입니다.",neutral
2,"이름부터 뭔가 거창한 이 기업, 최근 주가가 하늘 높은 줄 모르고 치솟고 있습니다.",neutral
3,도대체 무슨 일이 벌어지고 있는 걸까요?,neutral
4,주가는 거짓말을 안 한다?,neutral
...,...,...
56,"만약 그 질문에 ‘YES’라고 대답하신다면, 두산에너빌리티는 지금도 충분히 매력적인...",neutral
57,긴 글 읽어주셔서 감사합니다.,neutral
58,오늘도 현명한 투자되시길 바랍니다.,neutral
59,인플루언서 팬 하기 부탁드립니다,neutral


#### 3-4. 전체 7일간 포스팅 감성 분류
<p>서론, 혹은 이야기를 풀어나가기 위한 담화용 텍스트는 neutral로 분류되고 그 외 주가 판단에 대한 내용은 positive 혹은 negative로 분류되는 모습을 볼 수 있다.</p>
<p>이번에는 전체 70개 포스팅에 대해 모두 분류하고 분류 비중을 살펴보도록 하겠다.</p>
<p>포스팅을 먼저 모두 수집하고 전체 데이터에 대해 한 번에 추론하는 방식 혹은 매 포스팅마다 수집과 추론을 반복해서 결괏값을 합치는 방식을 취할 수 있다.</p>
<p>속도 및 작업 편의를 고려하면 전자가 좋겠지만 각 포스팅에 수십 개의 문장이 있기 때문에 전체 분류 타깃이 메모리에 한 번에 올라가게 되면 중간에 멈출 수도 있다. 따라서 배치 작업으로 안정적인 추론이 이루어질 수 있도록 후자 형태로 코드를 구성하겠다.</p>

In [29]:
for i in range(1, len(posts_info)):
    x = get_posts(posts_info[i])
    y = [x['label'] for x in senti_classifier(x)]
    df_next = pd.DataFrame(data={'text': x, 'senti': y})
    df = pd.concat([df, df_next])
    print(f'[INFO] 분류 작업 중.. (타겟 포스팅: {i}/{len(posts_info)-1} 분류 된 문장 수: {len(df)}')

[INFO] 분류 작업 중.. (타겟 포스팅: 1/69 분류 된 문장 수: 102
[INFO] 분류 작업 중.. (타겟 포스팅: 2/69 분류 된 문장 수: 181
[INFO] 분류 작업 중.. (타겟 포스팅: 3/69 분류 된 문장 수: 200
[INFO] 분류 작업 중.. (타겟 포스팅: 4/69 분류 된 문장 수: 236
[INFO] 분류 작업 중.. (타겟 포스팅: 5/69 분류 된 문장 수: 258
[INFO] 분류 작업 중.. (타겟 포스팅: 6/69 분류 된 문장 수: 287
[INFO] 분류 작업 중.. (타겟 포스팅: 7/69 분류 된 문장 수: 348
[INFO] 분류 작업 중.. (타겟 포스팅: 8/69 분류 된 문장 수: 389
[INFO] 분류 작업 중.. (타겟 포스팅: 9/69 분류 된 문장 수: 468
[INFO] 분류 작업 중.. (타겟 포스팅: 10/69 분류 된 문장 수: 487
[INFO] 분류 작업 중.. (타겟 포스팅: 11/69 분류 된 문장 수: 523
[INFO] 분류 작업 중.. (타겟 포스팅: 12/69 분류 된 문장 수: 545
[INFO] 분류 작업 중.. (타겟 포스팅: 13/69 분류 된 문장 수: 574
[INFO] 분류 작업 중.. (타겟 포스팅: 14/69 분류 된 문장 수: 635
[INFO] 분류 작업 중.. (타겟 포스팅: 15/69 분류 된 문장 수: 676
[INFO] 분류 작업 중.. (타겟 포스팅: 16/69 분류 된 문장 수: 755
[INFO] 분류 작업 중.. (타겟 포스팅: 17/69 분류 된 문장 수: 774
[INFO] 분류 작업 중.. (타겟 포스팅: 18/69 분류 된 문장 수: 810
[INFO] 분류 작업 중.. (타겟 포스팅: 19/69 분류 된 문장 수: 832
[INFO] 분류 작업 중.. (타겟 포스팅: 20/69 분류 된 문장 수: 861
[INFO] 분류 작업 중.. (타겟 포스팅: 21/69 분류 된 문장 수: 922
[INFO] 분류 작업 중.. (타겟 포

In [32]:
df = df.reset_index(drop=True)
df

,text,senti
0,요즘 주식시장 보면 마치 롤러코스터를 탄 듯한 종목들이 많습니다.,neutral
1,"그중에서도 유난히 고공비행 중인 종목이 하나 있는데요, 바로 두산에너빌리티입니다.",neutral
2,"이름부터 뭔가 거창한 이 기업, 최근 주가가 하늘 높은 줄 모르고 치솟고 있습니다.",neutral
3,도대체 무슨 일이 벌어지고 있는 걸까요?,neutral
4,주가는 거짓말을 안 한다?,neutral
...,...,...
2865,마치며..,neutral
2866,"지금 이 시점에서의 엔비디아 실적 발표는 단순한 기업 보고가 아니라, AI 산업 전...",neutral
2867,"어떤 숫자가 나올지, 그리고 시장은 어떻게 반응할지에 따라, 다음 투자 전략도 크게...",neutral
2868,"따라서 너무 짧은 시야로 보기보단, 변동성을 활용한 분할 매수 전략과 함께 중장기적...",neutral


#### 3-5. 결과

<p>결과는 중립 73%, 긍정 18%, 부정 8%로 현재 불안정한 시장 상황을 충분히 반영하고 있다.</p>
<p>사실, 이는 해석하기 나름이다. '여전히 긍정적인 전망을 하는 사람들이 더 많기 때문에 아직 바닥이 아니다'라고 할 수도 있고, '이제 바닥을 찍고 상승하려는 움직임이 보인다'라고 할 수도 있다.</p>
<p>따라서 이렇게 하나의 지표나 결과를 보고 전체 주식시장을 판단하는 것은 무리가 있겠지만 이러한 비율을 매주 추적하며 시계열로 분석한다면 더 유의미한 추세적 지표로 활용할 수 있을 것이다.</p>

In [ ]:
<p>출처: https://songseungwon.tistory.com/125 [관성을 이기는 데이터:티스토리]</p>